In [4]:
%matplotlib inline

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from math import *
import tensorflow as tf

from _context import dl_uncertainty
from dl_uncertainty.models import tf_utils
from dl_uncertainty.models.tf_utils import layers

In [5]:
x_ = np.array([[1,2],[3,4.0]])
#x = tf.constant(np.array([[1,2],[3,4.0]]), dtype=tf.float32)
x = tf.placeholder(tf.float32, shape=[None, 2])
sess = tf.Session()

## 1. Sampling random operations with no variables

### 1.1. Dropout

In [6]:
@layers.scoped
def dropout(x):
    return tf.nn.dropout(x, keep_prob=0.5)

ys = layers.sample(lambda x: dropout(x, name='do'), x, 100)
y_mean = tf.reduce_mean(ys, 0)  # should approximate x
ys2 = layers.sample(lambda x: dropout(x, name='do'), x, 8, examplewise=True)

In [7]:
sess.run(tf.global_variables_initializer())

print('x')
print(sess.run(x, {x:x_}))

print('')
print('dropout(x) sampling ([n,N,...]) (first 3 samples)')
s = sess.run(ys, {x:x_})
print(s[:3], '...')
print('shape:', s.shape)

print('')
print('Sample mean (estimation of x)')
m = sess.run(y_mean, {x:x_})
print(m)

print('')
print('Example-wise dropout(x) sampling (first 2 elements)')
s = sess.run(ys2, {x:x_})
print(s[:2], '...')
print('shape:', s.shape)

x
[[1. 2.]
 [3. 4.]]

dropout(x) sampling ([n,N,...]) (first 3 samples)
[[[2. 4.]
  [6. 0.]]

 [[2. 0.]
  [6. 8.]]

 [[0. 0.]
  [6. 8.]]] ...
shape: (100, 2, 2)

Sample mean (estimation of x)
[[0.92 2.08]
 [3.   4.64]]

Example-wise dropout(x) sampling (first 2 elements)
[[[0. 0.]
  [2. 0.]
  [2. 4.]
  [0. 0.]
  [0. 4.]
  [2. 4.]
  [2. 0.]
  [2. 4.]]

 [[6. 0.]
  [6. 0.]
  [0. 8.]
  [6. 8.]
  [0. 0.]
  [0. 0.]
  [6. 8.]
  [6. 0.]]] ...
shape: (2, 8, 2)


### 1.2. `tf.random_*`, `use_tf_loop=False`

In [8]:
@layers.scoped
def random(m, sd):
    return tf.random_normal(shape=tf.shape(m), mean=m, stddev=sd)

ys = layers.sample(lambda x: random(x, x), x, 100, examplewise=False, use_tf_loop=False)
y_mean = tf.reduce_mean(ys, 0)  # should approximate x

In [9]:
print('')
print('random_normal(x, x) sampling ([n,N,...]) (first 3 samples)')
s = sess.run(ys, {x:x_})
print(s[:3], '...')
print('shape:', s.shape)

print('')
print('Sample mean (estimation of x)')
m = sess.run(y_mean, {x:x_})
print(m)


random_normal(x, x) sampling ([n,N,...]) (first 3 samples)
[[[ 3.6866517   2.7205243 ]
  [ 2.9887357   4.385809  ]]

 [[ 2.0059853  -0.23315668]
  [ 7.309952    6.9326687 ]]

 [[ 3.2583785   3.2224627 ]
  [ 5.8586497  -2.437625  ]]] ...
shape: (100, 2, 2)

Sample mean (estimation of x)
[[0.9974589 2.0099893]
 [2.7311115 4.9470916]]


## 2. Sampling operations with variables

In [10]:
def init_func(*args, **kwargs): return tf.random_uniform([])

@layers.scoped
def rando(x):
    v = tf.get_variable("var", initializer=tf.random_uniform([]))
    #tf.get_variable_scope().reuse_variables()
    print(v)
    return v*tf.random_uniform(tf.shape(x))*tf.identity(x)

In [11]:
#trando = tf.make_template('rando', rando)
#rr = [tf.map_fn(rando, x) for _ in range(2)]
samp = layers.sample(rando, x, 100, use_tf_loop=False)

<tf.Variable 'sample_3/rando/var:0' shape=() dtype=float32_ref>


In [12]:
sess.run(tf.global_variables_initializer())

#rr_ = sess.run(rr)
samp_ = sess.run(samp, {x:x_})
print(samp_)

#for a in rr_:
#    print(a)

[[[7.20086843e-02 6.03298366e-01]
  [1.58329344e+00 6.96839571e-01]]

 [[8.63495916e-02 9.49326336e-01]
  [8.37193787e-01 4.34983134e-01]]

 [[5.75191736e-01 1.00737286e+00]
  [3.13730836e-01 1.25724757e+00]]

 [[4.38294530e-01 6.01901114e-01]
  [7.82516420e-01 5.96459150e-01]]

 [[2.92558402e-01 4.52507079e-01]
  [1.51970196e+00 1.24405229e+00]]

 [[1.99524179e-01 2.16668397e-01]
  [4.84762609e-01 7.40701675e-01]]

 [[5.35811663e-01 4.43903863e-01]
  [1.75915909e+00 1.01874180e-01]]

 [[2.73382753e-01 6.98756158e-01]
  [9.92775798e-01 4.85763103e-01]]

 [[2.57842869e-01 1.09541893e+00]
  [1.46425366e+00 8.66764426e-01]]

 [[5.80922902e-01 1.02473772e+00]
  [4.27783966e-01 1.77694786e+00]]

 [[2.40509603e-02 7.38586664e-01]
  [3.35955560e-01 1.65066516e+00]]

 [[4.94476944e-01 1.43475443e-01]
  [1.59319937e-01 1.45752978e+00]]

 [[1.65069893e-01 1.00445557e+00]
  [9.65704799e-01 7.43283033e-01]]

 [[5.04879713e-01 5.91784000e-01]
  [8.57184052e-01 7.48598576e-01]]

 [[3.78151357e-01 9.

In [13]:
# samples should be different because outputs come from different subgraphs with different values of "var"
assert np.any(samp_[0] != samp_[1])

In [14]:
v = tf.get_variable("var2", initializer=tf.random_uniform([]))

@layers.scoped
def wrapv(fn, x):
    print(x.shape)
    with tf.control_dependencies([tf.assign(v, x[0,0])]):
        return fn(v)
samp = layers.sample(lambda x: wrapv(rando, x), x, 10)

(20,)


ValueError: Index out of range using input dim 1; input has only 1 dims for 'sample_4/map/while/wrapv/strided_slice' (op: 'StridedSlice') with input shapes: [20], [2], [2], [2] and with computed input tensors: input[3] = <1 1>.

In [18]:
sess.run(tf.global_variables_initializer())
sess.run(samp)

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder' with dtype float and shape [2,2]
	 [[Node: Placeholder = Placeholder[dtype=DT_FLOAT, shape=[2,2], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'Placeholder', defined at:
  File "C:\Users\Ivan\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Ivan\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Ivan\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Ivan\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\Ivan\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "C:\Users\Ivan\Anaconda3\lib\site-packages\zmq\eventloop\ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "C

In [ ]:
a = tf.